In [1]:
import pandas as pd
import numpy as np

In [2]:
capes_dat = pd.read_csv('data/capes_data.csv')
capes_dat

,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,Study Hours per Week,Average Grade Expected,Average Grade Received,Evalulation URL
0,Butler Elizabeth Annette,AAS 10 - Intro/African-American Studies (A),SP23,66,48,93.5%,100.0%,2.80,A- (3.84),B+ (3.67),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
1,Butler Elizabeth Annette,AAS 170 - Legacies of Research (A),SP23,20,7,100.0%,100.0%,2.50,A- (3.86),A- (3.92),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
2,Jones Ian William Nasser,ANAR 111 - Foundations of Archaeology (A),SP23,16,3,100.0%,100.0%,3.83,B+ (3.67),NaN,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
3,Shtienberg Gilad,ANAR 115 - Coastal Geomorphology/Environ (A),SP23,26,6,100.0%,83.3%,3.83,B+ (3.50),B (3.07),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
4,Braswell Geoffrey E.,ANAR 155 - Stdy Abrd: Ancient Mesoamerica (A),SP23,22,9,100.0%,100.0%,5.17,A (4.00),A (4.00),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
...,...,...,...,...,...,...,...,...,...,...,...
63358,Li Huai,VIS 105D - Aesthetics/Chinese Calligraphy (A),SU07,17,13,100.0%,100.0%,2.33,A (4.00),NaN,https://cape.ucsd.edu/scripts/detailedStats.as...
63359,Guerrero Raul M.,VIS 106A - Painting: Image Making (A),SU07,16,14,92.9%,92.9%,7.21,A- (3.79),NaN,https://cape.ucsd.edu/scripts/detailedStats.as...
63360,Mangolte Babette,VIS 194S - Fantasy In Film (A),SU07,80,57,74.1%,26.4%,4.27,B (3.21),NaN,https://cape.ucsd.edu/scripts/detailedStats.as...
63361,Holland Nicole Murphy,VIS 22 - Formations of Modern Art (A),SU07,40,33,100.0%,96.7%,4.32,B+ (3.62),NaN,https://cape.ucsd.edu/scripts/detailedStats.as...


In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import time

try:
    import urllib.request as urllib2
except ImportError:
    import urllib2

In [4]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run without UI
chrome_options.add_argument("--disable-gpu")  # Disable GPU for headless environments
chrome_options.add_argument("--no-sandbox")  # Disable sandboxing
chrome_options.add_argument("--disable-dev-shm-usage") 

In [5]:
driver = webdriver.Chrome()

In [6]:
url = 'https://ucannualwage.ucop.edu/wage/'
driver.get(url)
time.sleep(1)

In [7]:
from selenium.webdriver.common.by import By

def get_df(location, last, first):
    #create empty dataframe to hold info
    new_df = pd.DataFrame(columns=['Year', 'Location', 'First Name', 'Last Name', 'Title', 'Gross Pay', 'Regular Pay', 'Overtime Pay', 'Other Pay'])
    
    #find the table in the html by xpath
    table_body = driver.find_element(By.XPATH, '//*[@id="list2"]')
    
    #go through all entries (some will not be relevant)
    entries = table_body.find_elements(By.TAG_NAME, 'tr')
    if len(entries) == 0: 
        #if there are no entries for the person, then their salary is not available. Create empty row
        new_row = {'Year':"", 'Location':location, 'First Name':first, 'Last Name':last, 'Title':"", 'Gross Pay':"", 'Regular Pay':"", 'Overtime Pay':"", 'Other Pay':""}
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True, sort=False)
    else:
        #if there are entries, then add a row
        for i in range(0, len(entries)):
            cols = entries[i].find_elements(By.TAG_NAME, 'td')
            table_row = ''

            #assign the column data to header names
            #if the page does not load quick enough, let the program sleep for 0.5 seconds.
            #continue to scrape data until found.
            while True:
                try:
                    year = cols[2].text
                    location = cols[3].text
                    firstname = cols[4].text
                    lastname = cols[5].text
                    title = cols[6].text
                    grossp = cols[7].text
                    regp = cols[8].text
                    overp = cols[9].text
                    otherp = cols[10].text
                    break;
                except:
                    time.sleep(1)
                    continue;
             
            #avoid duplicates by making sure the names match before adding to df
            if firstname.split(' ')[0] != first or lastname.split(' ')[0] != last:
                continue
            
            #create a new row with the above values and add to the temporary dataframe
            new_row = {'Year':year, 'Location':location, 'First Name':firstname, 'Last Name':lastname, 'Title':title, 'Gross Pay':grossp, 'Regular Pay':regp, 'Overtime Pay':overp, 'Other Pay':otherp}
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True, sort=False)
        
    #return the dataframe
    return new_df

In [38]:
from selenium.webdriver.common.by import By

def search_prof_salary(location, last, first, year):
    
    #input location into form (we are looking for San Diego only)
    grade_dropdown = Select(driver.find_element(By.ID, "location"))
    grade_dropdown.select_by_value(location)

    #input year into form
    year_dropdown = Select(driver.find_element(By.ID, 'year'))
    year_dropdown.select_by_value(year)
    
    #input lastname into form
    driver.find_element(By.ID, 'lastname').send_keys(last)
    
    #input firstname into form
    driver.find_element(By.ID, 'firstname').send_keys(first)
    
    driver.find_element(By.XPATH, '//*[@id="searchButton"]').click()
    
    #allow the data to load
    time.sleep(3)
    
    #clear the text boxes
    driver.find_element(By.ID, 'lastname').clear()
    driver.find_element(By.ID, 'firstname').clear()
    
    #now we scrape the table and return the dataframe
    return get_df(location, last, first)

In [9]:
rady_profs = capes_dat[capes_dat['Course'].str.contains('MGT')]
rady_profs

,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,Study Hours per Week,Average Grade Expected,Average Grade Received,Evalulation URL
761,Wilbur Kenneth C,MGT 100 - Customer Analytics (A),SP23,88,78,78.2%,79.5%,6.97,B (3.23),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
762,Wilbur Kenneth C,MGT 100 - Customer Analytics (B),SP23,89,83,77.1%,80.7%,6.99,B+ (3.36),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
763,Wilbur Kenneth C,MGT 100 - Customer Analytics (C),SP23,86,83,75.9%,71.1%,7.70,B+ (3.35),B (3.16),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
764,Ehrich Kristine R,MGT 103 - Product Marketing & Management (A),SP23,85,65,95.4%,93.8%,3.98,B+ (3.40),B (3.03),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
765,Ehrich Kristine R,MGT 103 - Product Marketing & Management (B),SP23,85,72,97.2%,98.6%,3.78,B+ (3.36),B (3.14),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
...,...,...,...,...,...,...,...,...,...,...,...
61777,Foit Delbert F.,MGT 121A - Innovation to Market A (A),WI08,64,44,95.3%,86.0%,3.20,A- (3.90),B+ (3.54),https://cape.ucsd.edu/scripts/detailedStats.as...
61778,August Terence,MGT 174 - Supply Chain & Operations Mgt (A),WI08,52,46,100.0%,100.0%,5.86,B+ (3.50),B (3.25),https://cape.ucsd.edu/scripts/detailedStats.as...
61779,Sullivan Robert S.,MGT 181 - Enterprise Finance (A),WI08,40,26,100.0%,100.0%,5.27,B (3.16),B (3.20),https://cape.ucsd.edu/scripts/detailedStats.as...
62754,Liersch Michael James,MGT 164 - Organizational Leadership (A),FA07,66,55,100.0%,100.0%,2.93,B+ (3.58),B+ (3.39),https://cape.ucsd.edu/scripts/detailedStats.as...


In [36]:
subset_2023 = rady_profs[rady_profs['Quarter'].str.contains('23')]
subset_2022 = rady_profs[rady_profs['Quarter'].str.contains('22')]
subset_2021 = rady_profs[rady_profs['Quarter'].str.contains('21')]
subset_2020 = rady_profs[rady_profs['Quarter'].str.contains('20')]

In [37]:
unique_instructors_2023 = subset_2023['Instructor'].unique()
unique_instructors_2022 = subset_2022['Instructor'].unique()
unique_instructors_2021 = subset_2021['Instructor'].unique()
unique_instructors_2020 = subset_2020['Instructor'].unique()

In [41]:
df = pd.DataFrame(unique_instructors_2023, columns=['Instructor'])
df_names = df.iloc[:, 0]

df_prof_salary_2023 = pd.DataFrame() #create empty dataframe

for name in df_names:
    chunks_fl = name.split(',') #split the name into first-middle and last
    chunks_lm = chunks_fl[0].split(' ') #split the last name
    chunks_fm = chunks_fl[-1].split(' ') #split the first name 

    last = chunks_lm[0].upper() #get only one last name
    first = chunks_fm[1].upper() #get only one first name
    
    df_prof_salary_2023 = pd.concat([df_prof_salary_2023, search_prof_salary("San Diego", last, first, '2023')], ignore_index=True)

In [42]:
df = pd.DataFrame(unique_instructors_2022, columns=['Instructor'])
df_names = df.iloc[:, 0]

df_prof_salary_2022 = pd.DataFrame() #create empty dataframe

for name in df_names:
    chunks_fl = name.split(',') #split the name into first-middle and last
    chunks_lm = chunks_fl[0].split(' ') #split the last name
    chunks_fm = chunks_fl[-1].split(' ') #split the first name 

    last = chunks_lm[0].upper() #get only one last name
    first = chunks_fm[1].upper() #get only one first name
    
    df_prof_salary_2022 = pd.concat([df_prof_salary_2022, search_prof_salary("San Diego", last, first, '2022')], ignore_index=True)

In [43]:
df = pd.DataFrame(unique_instructors_2021, columns=['Instructor'])
df_names = df.iloc[:, 0]

df_prof_salary_2021 = pd.DataFrame() #create empty dataframe

for name in df_names:
    chunks_fl = name.split(',') #split the name into first-middle and last
    chunks_lm = chunks_fl[0].split(' ') #split the last name
    chunks_fm = chunks_fl[-1].split(' ') #split the first name 

    last = chunks_lm[0].upper() #get only one last name
    first = chunks_fm[1].upper() #get only one first name
    
    df_prof_salary_2021 = pd.concat([df_prof_salary_2021, search_prof_salary("San Diego", last, first, '2021')], ignore_index=True)

In [44]:
df = pd.DataFrame(unique_instructors_2020, columns=['Instructor'])
df_names = df.iloc[:, 0]

df_prof_salary_2020 = pd.DataFrame() #create empty dataframe

for name in df_names:
    chunks_fl = name.split(',') #split the name into first-middle and last
    chunks_lm = chunks_fl[0].split(' ') #split the last name
    chunks_fm = chunks_fl[-1].split(' ') #split the first name 

    last = chunks_lm[0].upper() #get only one last name
    first = chunks_fm[1].upper() #get only one first name
    
    df_prof_salary_2020 = pd.concat([df_prof_salary_2020, search_prof_salary("San Diego", last, first, '2020')], ignore_index=True)

In [76]:
df_prof_salary = pd.concat([df_prof_salary_2023, df_prof_salary_2022, df_prof_salary_2021, df_prof_salary_2020])

In [77]:
df_prof_salary = df_prof_salary[(df_prof_salary['Gross Pay'] != '')]
df_prof_salary

,Year,Location,First Name,Last Name,Title,Gross Pay,Regular Pay,Overtime Pay,Other Pay
1,2023,San Diego,KRISTINE,EHRICH,LECT-AY-CONTINUING,"195,886.00","167,724.00",0.00,"28,162.00"
2,2023,San Diego,KENNETH,BATES,LECT-AY,"141,228.00","141,228.00",0.00,0.00
4,2023,San Diego,TYAGARAJAN,SOMASUNDARAM,VIS ASSOC PROF,"129,708.00","129,708.00",0.00,0.00
5,2023,San Diego,CARSTEN,ZIMMERMANN,VIS ASSOC PROF,"195,995.00","182,395.00",0.00,"13,600.00"
7,2023,San Diego,JON,ANDERSON,LECT-AY-1/9,"51,528.00","51,528.00",0.00,0.00
...,...,...,...,...,...,...,...,...,...
42,2020,San Diego,CHRISTOPHER,GOPAL,LECT IN SUMMER SESSION,"56,288.00","49,527.00",0.00,"6,761.00"
43,2020,San Diego,EDWIGE,CHEYNEL,ASST PROF-AY-B/E/E,"171,725.00","162,225.00",0.00,"9,500.00"
44,2020,San Diego,KEVIN,ZHU,PROF-AY-B/E/E,"289,553.00","222,503.00",0.00,"67,050.00"
45,2020,San Diego,JEREMY,BERTOMEU,ASSOC PROF-AY-B/E/E,"167,475.00","167,475.00",0.00,0.00


In [78]:
df_prof_salary.to_csv('prof_salary_data.csv', index=False)

In [79]:
subset_dat = pd.concat([subset_2023, subset_2022, subset_2021, subset_2020])
subset_dat.head()

,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,Study Hours per Week,Average Grade Expected,Average Grade Received,Evalulation URL
761,Wilbur Kenneth C,MGT 100 - Customer Analytics (A),SP23,88,78,78.2%,79.5%,6.97,B (3.23),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
762,Wilbur Kenneth C,MGT 100 - Customer Analytics (B),SP23,89,83,77.1%,80.7%,6.99,B+ (3.36),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
763,Wilbur Kenneth C,MGT 100 - Customer Analytics (C),SP23,86,83,75.9%,71.1%,7.70,B+ (3.35),B (3.16),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
764,Ehrich Kristine R,MGT 103 - Product Marketing & Management (A),SP23,85,65,95.4%,93.8%,3.98,B+ (3.40),B (3.03),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
765,Ehrich Kristine R,MGT 103 - Product Marketing & Management (B),SP23,85,72,97.2%,98.6%,3.78,B+ (3.36),B (3.14),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...


In [80]:
subset_dat.to_csv('rady_capes.csv', index=False)

In [81]:
subset_dat['last_name'] = subset_dat['Instructor'].str.split(" ").apply(lambda x: x[0].upper())
subset_dat['year_tail'] = subset_dat['Quarter'].str[2:]
subset_dat.head()

,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,Study Hours per Week,Average Grade Expected,Average Grade Received,Evalulation URL,last_name,year_tail
761,Wilbur Kenneth C,MGT 100 - Customer Analytics (A),SP23,88,78,78.2%,79.5%,6.97,B (3.23),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,WILBUR,23
762,Wilbur Kenneth C,MGT 100 - Customer Analytics (B),SP23,89,83,77.1%,80.7%,6.99,B+ (3.36),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,WILBUR,23
763,Wilbur Kenneth C,MGT 100 - Customer Analytics (C),SP23,86,83,75.9%,71.1%,7.70,B+ (3.35),B (3.16),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,WILBUR,23
764,Ehrich Kristine R,MGT 103 - Product Marketing & Management (A),SP23,85,65,95.4%,93.8%,3.98,B+ (3.40),B (3.03),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,EHRICH,23
765,Ehrich Kristine R,MGT 103 - Product Marketing & Management (B),SP23,85,72,97.2%,98.6%,3.78,B+ (3.36),B (3.14),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,EHRICH,23


In [82]:
df_prof_salary['year_tail'] = df_prof_salary['Year'].str[2:]
df_prof_salary.head()

,Year,Location,First Name,Last Name,Title,Gross Pay,Regular Pay,Overtime Pay,Other Pay,year_tail
1,2023,San Diego,KRISTINE,EHRICH,LECT-AY-CONTINUING,"195,886.00","167,724.00",0.00,"28,162.00",23
2,2023,San Diego,KENNETH,BATES,LECT-AY,"141,228.00","141,228.00",0.00,0.00,23
4,2023,San Diego,TYAGARAJAN,SOMASUNDARAM,VIS ASSOC PROF,"129,708.00","129,708.00",0.00,0.00,23
5,2023,San Diego,CARSTEN,ZIMMERMANN,VIS ASSOC PROF,"195,995.00","182,395.00",0.00,"13,600.00",23
7,2023,San Diego,JON,ANDERSON,LECT-AY-1/9,"51,528.00","51,528.00",0.00,0.00,23


In [83]:
capes_salary_merge = subset_dat.merge(df_prof_salary, left_on=['last_name', 'year_tail'], right_on=['Last Name', 'year_tail'], how='left')
capes_salary_merge

,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,Study Hours per Week,Average Grade Expected,Average Grade Received,...,year_tail,Year,Location,First Name,Last Name,Title,Gross Pay,Regular Pay,Overtime Pay,Other Pay
0,Wilbur Kenneth C,MGT 100 - Customer Analytics (A),SP23,88,78,78.2%,79.5%,6.97,B (3.23),B (3.15),...,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wilbur Kenneth C,MGT 100 - Customer Analytics (B),SP23,89,83,77.1%,80.7%,6.99,B+ (3.36),B (3.15),...,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wilbur Kenneth C,MGT 100 - Customer Analytics (C),SP23,86,83,75.9%,71.1%,7.70,B+ (3.35),B (3.16),...,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ehrich Kristine R,MGT 103 - Product Marketing & Management (A),SP23,85,65,95.4%,93.8%,3.98,B+ (3.40),B (3.03),...,23,2023,San Diego,KRISTINE,EHRICH,LECT-AY-CONTINUING,"195,886.00","167,724.00",0.00,"28,162.00"
4,Ehrich Kristine R,MGT 103 - Product Marketing & Management (B),SP23,85,72,97.2%,98.6%,3.78,B+ (3.36),B (3.14),...,23,2023,San Diego,KRISTINE,EHRICH,LECT-AY-CONTINUING,"195,886.00","167,724.00",0.00,"28,162.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,Wagner Timothy Ryan,MGT 3 - Quant Methods in Business (A),WI20,53,26,88.0%,88.0%,4.26,B+ (3.59),B (3.21),...,20,2020,San Diego,TIMOTHY,WAGNER,LECT-AY,"34,493.00","34,493.00",0.00,0.00
992,Levkoff Steven B.,MGT 4 - Financial Accounting (A),WI20,9,3,66.7%,66.7%,5.17,C+ (2.67),NaN,...,20,2020,San Diego,STEVEN,LEVKOFF,LECT-AY-CONTINUING,"111,044.00","88,717.00",0.00,"22,327.00"
993,Pecore Joseph P.,MGT 45 - Principles of Accounting (A),WI20,141,131,96.9%,96.9%,5.75,B+ (3.47),B+ (3.34),...,20,2020,San Diego,JOSEPH,PECORE,LECT-AY-CONTINUING,"188,561.00","174,103.00",0.00,"14,458.00"
994,Houskeeper Robert,MGT 5 - Managerial Accounting (A),WI20,97,74,83.6%,83.6%,4.61,B (3.16),B- (2.92),...,20,2020,San Diego,ROBERT,HOUSKEEPER,LECT-AY-CONTINUING,"163,335.00","139,603.00",0.00,"23,732.00"


In [84]:
capes_salary_merge.to_csv('merged_salary_capes.csv', index=False)

In [5]:
rady_dat = pd.read_csv('data/rady_capes.csv')
salary_dat = pd.read_csv('data/prof_salary_data.csv')

In [10]:
rady_dat

,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,Study Hours per Week,Average Grade Expected,Average Grade Received,Evalulation URL
0,Wilbur Kenneth C,MGT 100 - Customer Analytics (A),SP23,88,78,78.2%,79.5%,6.97,B (3.23),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
1,Wilbur Kenneth C,MGT 100 - Customer Analytics (B),SP23,89,83,77.1%,80.7%,6.99,B+ (3.36),B (3.15),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
2,Wilbur Kenneth C,MGT 100 - Customer Analytics (C),SP23,86,83,75.9%,71.1%,7.70,B+ (3.35),B (3.16),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
3,Ehrich Kristine R,MGT 103 - Product Marketing & Management (A),SP23,85,65,95.4%,93.8%,3.98,B+ (3.40),B (3.03),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
4,Ehrich Kristine R,MGT 103 - Product Marketing & Management (B),SP23,85,72,97.2%,98.6%,3.78,B+ (3.36),B (3.14),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
...,...,...,...,...,...,...,...,...,...,...,...
876,Wagner Timothy Ryan,MGT 3 - Quant Methods in Business (A),WI20,53,26,88.0%,88.0%,4.26,B+ (3.59),B (3.21),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
877,Levkoff Steven B.,MGT 4 - Financial Accounting (A),WI20,9,3,66.7%,66.7%,5.17,C+ (2.67),NaN,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
878,Pecore Joseph P.,MGT 45 - Principles of Accounting (A),WI20,141,131,96.9%,96.9%,5.75,B+ (3.47),B+ (3.34),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
879,Houskeeper Robert,MGT 5 - Managerial Accounting (A),WI20,97,74,83.6%,83.6%,4.61,B (3.16),B- (2.92),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...
